<h1>Two example of load and preprocess text</h1>

# Import library


In [1]:
import collections
import pathlib
import re
import string

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds

In [2]:
!pip install tensorflow-text

     |████████████████████████████████| 2.6MB 9.8MB/s 


In [3]:
import tensorflow_text as tf_text

# Example 1: Predict the tag for a Stack Overflow question

As a first example, you will download a dataset of programming questions from Stack Overflow. Each question ("How do I sort a dictionary by value?") is labeled with exactly one tag (Python, CSharp, JavaScript, or Java). Your task is to develop a model that predicts the tag for a question. This is an example of multi-class classification, an important and widely applicable kind of machine learning problem.

## Download and explore the dataset

In [4]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
dataset = utils.get_file(
    'stack_overflow_16k.tar.gz',
    data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir='')
dataset_dir = pathlib.Path(dataset).parent

6053888/6053168 [==============================] - 0s 0us/step


In [5]:
list(dataset_dir.iterdir())

[PosixPath('/tmp/.keras/stack_overflow_16k.tar.gz.tar.gz'),
 PosixPath('/tmp/.keras/README.md'),
 PosixPath('/tmp/.keras/train'),
 PosixPath('/tmp/.keras/test')]

In [6]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[PosixPath('/tmp/.keras/train/csharp'),
 PosixPath('/tmp/.keras/train/javascript'),
 PosixPath('/tmp/.keras/train/python'),
 PosixPath('/tmp/.keras/train/java')]

*The train/csharp, train/java, train/python and train/javascript directories contain many text files, each of which is a Stack Overflow question. Print a file and inspect the data.*

In [7]:
sample_file = train_dir/'python/1755.txt'
with open(sample_file) as f:
  print(f.read())

why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



## Load the dataset

When running a machine learning experiment, it is a best practice to divide your dataset into three splits: train, validation, and test. The Stack Overflow dataset has already been divided into train and test, but it lacks a validation set. Create a validation set using an 80:20 split of the training data by using the validation_split argument below.

In [8]:
batch_size = 32
seed = 42

raw_train_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


*As you can see above, there are 8,000 examples in the training folder, of which you will use 80% (or 6,400) for training. As you will see in a moment, you can train a model by passing a tf.data.Dataset directly to model.fit. First, iterate over the dataset and print out a few examples, to get a feel for the data.*

In [9]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(10):
    print("Question: ", text_batch.numpy()[i][:100], '...')
    print("Label:", label_batch.numpy()[i])

Question:  b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can' ...
Label: 1
Question:  b'"blank code slow skin detection this code changes the color space to lab and using a threshold finds' ...
Label: 3
Question:  b'"option and validation in blank i want to add a new option on my system where i want to add two text' ...
Label: 1
Question:  b'"exception: dynamic sql generation for the updatecommand is not supported against a selectcommand th' ...
Label: 0
Question:  b'"parameter with question mark and super in blank, i\'ve come across a method that is formatted like t' ...
Label: 1
Question:  b'call two objects wsdl the first time i got a very strange wsdl. ..i would like to call the object (i' ...
Label: 0
Question:  b'how to correctly make the icon for systemtray in blank using icon sizes of any dimension for systemt' ...
Label: 0
Question:  b'"is there a way to check a variable that exists in a different script than the original on

In [10]:
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


In [11]:
raw_val_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [12]:
test_dir = dataset_dir/'test'
raw_test_ds = preprocessing.text_dataset_from_directory(
    test_dir, batch_size=batch_size)

Found 8000 files belonging to 4 classes.


## Prepare the dataset for training

Next, you will **standardize, tokenize,** and **vectorize** the data using the preprocessing.TextVectorization layer

In [13]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

For int mode, in addition to maximum vocabulary size, you need to set an explicit maximum sequence length, which will cause the layer to pad or truncate sequences to exactly sequence_length values.

In [14]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

**Next, you will call adapt to fit the state of the preprocessing layer to the dataset. This will cause the model to build an index of strings to integers.**

In [15]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

**See the result of using these layers to preprocess data:**

In [16]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label
  
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [17]:
# Retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

Question tf.Tensor(b'"function expected error in blank for dynamically created check box when it is clicked i want to grab the attribute value.it is working in ie 8,9,10 but not working in ie 11,chrome shows function expected error..&lt;input type=checkbox checked=\'checked\' id=\'symptomfailurecodeid\' tabindex=\'54\' style=\'cursor:pointer;\' onclick=chkclickevt(this);  failurecodeid=""1"" &gt;...function chkclickevt(obj) { .    alert(obj.attributes(""failurecodeid""));.}"\n', shape=(), dtype=string)
Label tf.Tensor(2, shape=(), dtype=int32)


In [18]:
print("'binary' vectorized question:", 
      binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor([[1. 1. 1. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [19]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])

'int' vectorized question: tf.Tensor(
[[  38  450   65    7   16   12  892  265  186  451   44   11    6  685
     3   46    4 2062    2  485    1    6  158    7  479    1   26   20
   158    7  479    1  502   38  450    1 1767 1763    1    1    1    1
     1    1    1    1    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0

**You can lookup the token (string) that each integer corresponds to by calling .get_vocabulary() on the layer.**

In [20]:
print("1289 ---> ", int_vectorize_layer.get_vocabulary()[1289])
print("313 ---> ", int_vectorize_layer.get_vocabulary()[313])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

1289 --->  roman
313 --->  source
Vocabulary size: 10000


**You are nearly ready to train your model. As a final preprocessing step, you will apply the TextVectorization layers you created earlier to the train, validation, and test dataset.**

In [21]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

## Configure the dataset for performance

In [22]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)
  
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

## Train the model

**It's time to create our neural network. For the binary vectorized data, train a simple bag-of-words linear model:**

In [23]:
binary_model = tf.keras.Sequential([layers.Dense(4)])
binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)

Epoch 1/10
200/200 [==============================] - 3s 13ms/step - loss: 1.1192 - accuracy: 0.6514 - val_loss: 0.9129 - val_accuracy: 0.7750
Epoch 2/10
200/200 [==============================] - 1s 3ms/step - loss: 0.7776 - accuracy: 0.8205 - val_loss: 0.7489 - val_accuracy: 0.7994
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 0.6265 - accuracy: 0.8633 - val_loss: 0.6634 - val_accuracy: 0.8112
Epoch 4/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5334 - accuracy: 0.8892 - val_loss: 0.6101 - val_accuracy: 0.8256
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4675 - accuracy: 0.9056 - val_loss: 0.5735 - val_accuracy: 0.8325
Epoch 6/10
200/200 [==============================] - 1s 3ms/step - loss: 0.4173 - accuracy: 0.9172 - val_loss: 0.5469 - val_accuracy: 0.8363
Epoch 7/10
200/200 [==============================] - 0s 2ms/step - loss: 0.3772 - accuracy: 0.9287 - val_loss: 0.5268 - val_accuracy: 0.8406
Epoch

**Next, you will use the int vectorized layer to build a 1D ConvNet.**

In [24]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [25]:
# vocab_size is VOCAB_SIZE + 1 since 0 is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=10)

Epoch 1/10
200/200 [==============================] - 4s 19ms/step - loss: 1.1383 - accuracy: 0.5233 - val_loss: 0.7411 - val_accuracy: 0.7013
Epoch 2/10
200/200 [==============================] - 2s 10ms/step - loss: 0.6050 - accuracy: 0.7675 - val_loss: 0.5326 - val_accuracy: 0.8012
Epoch 3/10
200/200 [==============================] - 2s 10ms/step - loss: 0.3611 - accuracy: 0.8889 - val_loss: 0.4706 - val_accuracy: 0.8188
Epoch 4/10
200/200 [==============================] - 2s 10ms/step - loss: 0.1991 - accuracy: 0.9553 - val_loss: 0.4669 - val_accuracy: 0.8169
Epoch 5/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0980 - accuracy: 0.9844 - val_loss: 0.4886 - val_accuracy: 0.8188
Epoch 6/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0443 - accuracy: 0.9958 - val_loss: 0.5179 - val_accuracy: 0.8219
Epoch 7/10
200/200 [==============================] - 2s 9ms/step - loss: 0.0209 - accuracy: 0.9995 - val_loss: 0.5453 - val_accuracy: 0.8188


## Compare the two models

In [28]:
print("Linear model on binary vectorized data:")
print(binary_model.summary())

Linear model on binary vectorized data:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())

ConvNet model on int vectorized data:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))

250/250 [==============================] - 2s 10ms/step - loss: 0.6469 - accuracy: 0.8033
Binary model accuracy: 81.59%
Int model accuracy: 80.33%


## Export the model

In [31]:
export_model = tf.keras.Sequential(
    [binary_vectorize_layer, binary_model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(binary_accuracy))

250/250 [==============================] - 3s 11ms/step - loss: 0.7075 - accuracy: 0.8159
Accuracy: 81.59%


***Now your model can take raw strings as input and predict a score for each label using model.predict. Define a function to find the label with the maximum score:***

In [32]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels

## Run inference on new data

In [36]:
inputs = [
    "how do I extract keys from a dict into a list?",  # python
    "debug public static void main(string[] args) {...}",  # java
    "panda library for data analytics",
]
predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)
for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())

Question:  how do I extract keys from a dict into a list?
Predicted label:  b'python'
Question:  debug public static void main(string[] args) {...}
Predicted label:  b'java'
Question:  panda library for data analytics
Predicted label:  b'python'
